# Object Classifier

## Process images

### Extract data from .7z files

In [ ]:
import os

os.mkdir("data")


In [ ]:
from zipfile import ZipFile
dataset = ZipFile('data/cifar-10.zip')
dataset.extractall(path='data')
dataset.close()
print("Dataset extracted successfully")

Dataset extracted successfully


In [ ]:
import py7zr

archive = py7zr.SevenZipFile('data/train.7z', mode='r')
archive.extractall(path='data')
archive.close()
train_file_location='data/train'

archive = py7zr.SevenZipFile('data/test.7z', mode='r')
archive.extractall(path='data')
archive.close()
test_file_location='data/test'


In [3]:
#imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [4]:
filenames = os.listdir("data/train")
print(len(filenames))

50000


### Process Data

In [15]:
def get_image_as_array(path):
    try:
        image = Image.open(path)
        return np.array(image)
    except:
        return None

df = pd.read_csv('data/trainLabels.csv')
df['image'] = df['id'].apply(lambda x: get_image_as_array(f'data/train/{x}.png'))





In [16]:
df.head()

,id,label,image
0,1,frog,"[[[59, 62, 63], [43, 46, 45], [50, 48, 43], [6..."
1,2,truck,"[[[154, 177, 187], [126, 137, 136], [105, 104,..."
2,3,truck,"[[[255, 255, 255], [253, 253, 253], [253, 253,..."
3,4,deer,"[[[28, 25, 10], [37, 34, 19], [38, 35, 20], [4..."
4,5,automobile,"[[[170, 180, 198], [168, 178, 196], [177, 185,..."


In [17]:
df.dropna(inplace=True, subset=['image'])

In [ ]:
df.shape

(49999, 3)

In [19]:
object_classes = df['label'].unique()
print(object_classes)

['frog' 'truck' 'deer' 'automobile' 'bird' 'horse' 'ship' 'cat' 'dog'
 'airplane']


In [20]:


entries_per_class = df['label'].value_counts()
print(entries_per_class) # no need to standardize or scale

label
frog          5000
truck         5000
deer          5000
bird          5000
horse         5000
ship          5000
cat           5000
dog           5000
airplane      5000
automobile    4999
Name: count, dtype: int64


In [21]:
# labels_dictionary = dict(enumerate(object_classes))
# print(labels_dictionary)
labels_map = {}
for index, item in enumerate(object_classes):
    labels_map[item] = index

print(labels_map)

{'frog': 0, 'truck': 1, 'deer': 2, 'automobile': 3, 'bird': 4, 'horse': 5, 'ship': 6, 'cat': 7, 'dog': 8, 'airplane': 9}


In [22]:
df['label'] = df['label'].map(labels_map)
df['image'] = df['image']/255
df.head()
df.shape

(49999, 3)

In [23]:
num_rows_80_percent = int(len(df) * 0.8)

train = df.iloc[:num_rows_80_percent]
test = df.iloc[num_rows_80_percent:]

print(train.shape)
print(test.shape)


(39999, 3)
(10000, 3)


In [27]:
X_train = np.array(train['image'])
Y_train = np.array(train['label'])

X_test = np.array(test['image'])
Y_test = np.array(test['label'])

print("X_train.shape: ", X_train.shape)
print("Y_train.shape: ", Y_train.shape)

print("X_test.shape: ", X_test.shape)
print("Y_test.shape: ", Y_test.shape)





X_train.shape:  (39999,)
Y_train.shape:  (39999,)
X_test.shape:  (10000,)
Y_test.shape:  (10000,)


## Create Neural Net

We shall create the Neural Network

In [28]:
import tensorflow as tf
from tensorflow import keras



/Users/ansinha/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [29]:
num_of_classes = len(object_classes)

#setting up layers

inputs = keras.Input(shape=(32, 32, 3))
x = keras.layers.Flatten()(inputs)
x = keras.layers.Dense(64, activation='relu')(x)
outputs = keras.layers.Dense(num_of_classes, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)